In [13]:
import numpy as np
import pandas as pd
import random
from keras.models import Sequential
from keras.layers import Dense

In [15]:
df = pd.read_csv('Bank_Personal_Loan_Modelling.csv')
df = df.drop(['ID', 'ZIP Code'], axis=1)
df = pd.get_dummies(df, columns=['Education', 'Personal Loan', 'Securities Account', 'CD Account', 'Online', 'CreditCard'],drop_first=True)
x = df.drop('Personal Loan_1', axis=1)
y = df['Personal Loan_1']

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [18]:
def create_model(individual):
    model = Sequential()
    model.add(Dense(units=individual[0], activation='relu', input_dim=x_train.shape[1]))
    for i in range(1, len(individual)):
        model.add(Dense(units=individual[i], activation='relu'))
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [19]:
def evaluate_model(individual):
    model = create_model(individual)
    model.fit(x_train, y_train, epochs=10, batch_size=32, verbose=0)
    _, accuracy = model.evaluate(x_train, y_train, verbose=0)
    return accuracy,

In [20]:
population_size = 10
n_generations = 5
n_features = x_train.shape[1]
n_hidden_layers = 2
min_neurons = 5
max_neurons = 50
mutation_prob = 0.1

In [21]:
population = []
for i in range(population_size):
    individual = []
    for j in range(n_hidden_layers):
        n_neurons = random.randint(min_neurons, max_neurons)
        individual.append(n_neurons)
    population.append(individual)

In [22]:
for gen in range(n_generations):
    print(f'Generation {gen+1}')
    fitness_values = []
    for individual in population:
        fitness_value = evaluate_model(individual)
        fitness_values.append(fitness_value)
    print(fitness_values)
    fitness_values = np.array(fitness_values)
    best_individual_idx = np.argmax(fitness_values)
    best_individual = population[best_individual_idx]
    print(f'Best individual: {best_individual}')
    new_population = [best_individual]
    for i in range(1, population_size):
        parent1_idx = np.random.randint(0, population_size)
        parent2_idx = np.random.randint(0, population_size)
        parent1 = population[parent1_idx]
        parent2 = population[parent2_idx]
        crossover_point = np.random.randint(1, n_hidden_layers)
        child = parent1[:crossover_point] + parent2[crossover_point:]
    for j in range(len(child)):
        if np.random.rand() < mutation_prob:
            child[j] = random.randint(min_neurons, max_neurons)
            new_population.append(child)
            population = new_population
                                    


Generation 1
[(0.9858666658401489,), (0.977066695690155,), (0.9794666767120361,), (0.9858666658401489,), (0.9855999946594238,), (0.9847999811172485,), (0.9714666604995728,), (0.9832000136375427,), (0.9746666550636292,), (0.9845333099365234,)]
Best individual: [50, 35]
Generation 2
[(0.9834666848182678,), (0.9781333208084106,), (0.9815999865531921,), (0.9842666387557983,), (0.9864000082015991,), (0.9853333234786987,), (0.9682666659355164,), (0.9837333559989929,), (0.9730666875839233,), (0.986133337020874,)]
Best individual: [40, 32]
Generation 3
[(0.986133337020874,), (0.9768000245094299,), (0.9794666767120361,), (0.9845333099365234,), (0.9847999811172485,), (0.9847999811172485,), (0.9664000272750854,), (0.981333315372467,), (0.9696000218391418,), (0.9837333559989929,)]
Best individual: [50, 35]
Generation 4
[(0.9869333505630493,), (0.9757333397865295,), (0.9815999865531921,), (0.9864000082015991,), (0.9847999811172485,), (0.9824000000953674,), (0.9677333235740662,), (0.9834666848182678

In [23]:
best_model = create_model(best_individual)
best_model.fit(x_train, y_train, epochs=10, batch_size=32, verbose=1)

Epoch 1/10
118/118 [==============================] - 1s 2ms/step - loss: 0.2967 - accuracy: 0.9027
Epoch 2/10
118/118 [==============================] - 0s 2ms/step - loss: 0.1317 - accuracy: 0.9483
Epoch 3/10
118/118 [==============================] - 0s 2ms/step - loss: 0.0941 - accuracy: 0.9672
Epoch 4/10
118/118 [==============================] - 0s 2ms/step - loss: 0.0779 - accuracy: 0.9749
Epoch 5/10
118/118 [==============================] - 0s 2ms/step - loss: 0.0697 - accuracy: 0.9768
Epoch 6/10
118/118 [==============================] - 0s 2ms/step - loss: 0.0644 - accuracy: 0.9795
Epoch 7/10
118/118 [==============================] - 0s 2ms/step - loss: 0.0600 - accuracy: 0.9800
Epoch 8/10
118/118 [==============================] - 0s 2ms/step - loss: 0.0581 - accuracy: 0.9811
Epoch 9/10
118/118 [==============================] - 0s 2ms/step - loss: 0.0541 - accuracy: 0.9813
Epoch 10/10
118/118 [==============================] - 0s 2ms/step - loss: 0.0532 - accuracy: 0.9811

In [24]:
_, test_accuracy = best_model.evaluate(x_test, y_test, verbose=0)
print(f'Accuracy: {test_accuracy}')

Accuracy: 0.9783999919891357
